## notes  
4/4/2025
Here I will annotate the mouse data per stage. First I will just do a rough annotation to facilitate SAMap.  
Later on I will go into more detail.  

5/5/2025  
Had to take a little break to work on integrating the olfactory derived ectoderm for andrea. I made some changes there to the RNA velocity which I think will work well.  
I upgraded the interactive viewer to allow changes to the h5ad file and creation of dotplots. I also did my best to clean up the cartilage shark data for Elio.  

Now continuing with mouse. After the stages I used for SAMap to compare with shark, I continue with the annotation of E10/11/12.

In [ ]:
import logging
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)
import scanpy as sc
import scvelo as scv
import scvi
import seaborn as sns
import plotly.express as px
import numpy as np
from dash import Dash, dcc, html, Input, Output,dash_table, ctx, State
import dash_ag_grid as dag

import pandas as pd

import os
import sys
import time
import gc
os.environ['R_HOME'] = sys.exec_prefix+"/lib/R/"

# Plotting
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
from matplotlib.lines import Line2D 

from copy import copy
reds = copy(mpl.cm.Reds)
reds.set_under("lightgray")

def matplotlib_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []

    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k * h)[:3]) * 255))
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])

    return pl_colorscale

plotly_reds = matplotlib_to_plotly(reds, 255)
plotly_reds[0] = [0.0, 'rgb(211, 211, 211)']

project_directory = '/Cranio_Lab/Louk_Seton/4_species_project'
os.chdir(os.path.expanduser("~")+project_directory)

In [ ]:
##read in all samples
sample_names = ['ME8','ME9','ME10','ME11','ME12'] #specify the sample names
species = 'mouse' #specify the species
genome = 'mm10' #specify the genome
output_prefix = 'h5ad_files/' #specify the location of the cellranger output

adata_dict = {}
for sample in sample_names:
    adata_dict[sample] = sc.read(output_prefix+species+'/'+genome+'/'+sample+'_after_filtering.h5ad')

In [ ]:
leiden_res = [0.5,0.6,0.7,0.8,0.9,1,1.2,1.4,1.6,1.8,2,2.2,2.4,2.6,2.8,3]
for sample in adata_dict:
    for i in leiden_res:
        sc.tl.leiden(adata_dict[sample], resolution = i, key_added = 'leiden'+str(i))
    try:
        del adata_dict[sample].uns['log1p']
    except: print('no log1p')
    for i in leiden_res:
        sc.tl.rank_genes_groups(adata_dict[sample], 'leiden'+str(i), method='wilcoxon', 
                                key_added = "wilcoxon_"+'leiden'+str(i))

    sc.tl.rank_genes_groups(adata_dict[sample], 'leiden_post_QC', method='wilcoxon', 
                            key_added = "wilcoxon_"+'leiden_post_QC')
    adata_dict[sample].write(output_prefix+species+'/'+genome+'/'+sample+'_interactive.h5ad')

In [ ]:
## E8
adata = adata_dict['ME8']

In [ ]:
sc.tl.leiden(adata, resolution = 4, key_added = 'leiden_high')

In [ ]:
sc.pl.umap(adata, color = ['leiden_post_QC',],
           #groups = ['35'],
           legend_loc = 'on data',
           ncols = 1)

In [ ]:
sc.pl.umap(adata,color = ['Cdx1','Hes7','Hoxb8','Hoxb9','L1td1','Xbp1','Pmp22'],
           ncols = 3,cmap=reds, vmin =0.05)

In [ ]:
sc.tl.rank_genes_groups(adata, groupby = 'leiden_post_QC', method = 'wilcoxon')

In [ ]:
sc.pl.rank_genes_groups_dotplot(adata, n_genes=4,
                                values_to_plot="logfoldchanges", cmap='bwr',
                                vmin=-4, vmax=4,
                                min_logfoldchange=3,
                                colorbar_title='log fold change'
                               )

In [ ]:
sc.pl.rank_genes_groups_dotplot(adata, n_genes=40,
                                groups = ['19'],
                                values_to_plot="logfoldchanges", cmap='bwr',
                                vmin=-4, vmax=4,
                                min_logfoldchange=4,
                                colorbar_title='log fold change'
                               )

In [ ]:
#quick annotation for E8
cluster_anno = {
    '0':'Mesoderm - Cranial - Foxl2+',# seems to be anterior mesoderm https://www.cell.com/developmental-cell/fulltext/S1534-5807%2820%2930889-3
    '1':'Forebrain - Six3+',
    '2':'Midbrain - Pax5+',
    '3':'Neural Tube - Ccdc198+', #is 1700011H14Rik, also Cpn1 in same area in in situ https://bmcdevbiol.biomedcentral.com/articles/10.1186/1471-213X-7-92/tables/2
    '4':'Surface Ectoderm - Cranial - Epcam+',
    '5':'Neural Crest - Sox10+',
    '6':'Myocyte - Foxf1+', #could be myotome or lateral plate mesoderm. I don't know. Similar to heart but lacking heart markers. Has muscle markers.
    '7':'Somite - Meox1+',
    '8':'Hindbrain - Hoxa2+',
    '9':'Endoderm - Pax9+',
    '10':'Mesoderm - Pharyngeal - Tlx1+',
    '11':'Hindbrain - Vgll3+', #either rhombomere 1 or 2, difficult to say
    '12':'Endothelium - Tal1+',
    '13':'Floor Plate/Ventral Neural Tube - Nkx2-9+', #Nkx2-9, Shh
    '14':'Surface Ectoderm - Pharyngeal - Tbx1+',
    '15':'Myocyte - Heart - Myh6+',
    '16':'Strange cells - Pou5f1+', #Pou5f1 high but half of them have Hox8/9 expression. And these two groups stick together regardless of resolution.
    '17':'Mesoderm - Tbx6+',#strange, has tail markers, not sure what to think
    '18':'Notochord - T+',
    '19':'Surface Ectoderm - Wnt3',
}
adata.obs['annotation'] = adata.obs['leiden_post_QC'].map(cluster_anno)

adata.obs["annotation_ordered"] = pd.Categorical(
    values=adata.obs.annotation, 
    categories=['Mesoderm - Cranial - Foxl2+',
                'Mesoderm - Pharyngeal - Tlx1+',
                'Somite - Meox1+',
                'Mesoderm - Tbx6+',
                'Myocyte - Foxf1+',
                'Myocyte - Heart - Myh6+', 
                'Endothelium - Tal1+',
                'Forebrain - Six3+', 
                'Midbrain - Pax5+',
                'Floor Plate/Ventral Neural Tube - Nkx2-9+',
                'Hindbrain - Vgll3+',
                'Hindbrain - Hoxa2+',
                'Neural Tube - Ccdc198+',
                'Neural Crest - Sox10+',
                'Strange cells - Pou5f1+',
                'Surface Ectoderm - Cranial - Epcam+',
                'Surface Ectoderm - Pharyngeal - Tbx1+',
                'Surface Ectoderm - Wnt3',
                'Endoderm - Pax9+',
                'Notochord - T+',
               ], 
    ordered=True
)

adata.uns['annotation_ordered_colors'] = ["#bc7253",
"#d0da3f",
"#d54f2d",
"#929632",
"#d79636",
"#826f32",
"#dacd81"] + ["#36dee6",
"#9c31df",
"#45c6a1",
"#6145c5",
"#7482d8",
"#cf6bd8",
"#3d2a7a",
"#8a3f90"] +["#8c1543",
"#ff79eb",
"#eb0078",
"#f898df",
"#ba00a6"]


In [ ]:
sc.pl.umap(adata,color = ['annotation_ordered'])

In [ ]:
adata.write(output_prefix+species+'/'+genome+'/'+'ME8_annotated.h5ad')
adata_dict['ME8'] = adata

In [ ]:
#ME9
adata = adata_dict['ME9']
sc.tl.leiden(adata, resolution = 3, key_added = 'leiden_high')

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

In [ ]:
sc.pl.umap(adata, color = ['leiden_post_QC',],
           #groups = ['35'],
           legend_loc = 'on data',
           ncols = 1)

In [ ]:

sc.pl.umap(adata,color = ['Casr','Krt8','Vax1','Hmx1','Aldh1a7','Pax6','Six1','Aldh1a3','Rax','Vsx2','Pax2','Otx2','Six3','Thrb','Lgr5','Slc44a5'],
           #['Mab21l2','Gbx2','Phox2a','Phox2b','Shh','Nkx2-1','Nkx2-4','Mitf','Vax2','Pax6','Emx2','Six6','Six3','Rspo2','Rspo3','Lhx5','Lhx2','Foxg1','Hoxa2','Otx2','Tal2','Pax3','Vgll3','Tmem255a','Plk3','Myt1','Neurod1','Aldh1a2','Dlx1','Dlx5','Trp63'],
           ncols = 3,cmap=reds, vmin =0.05)

In [ ]:
sc.tl.rank_genes_groups(adata, groupby = 'leiden_post_QC', method = 'wilcoxon')

In [ ]:
sc.pl.rank_genes_groups_dotplot(adata, n_genes=4,
                                values_to_plot="logfoldchanges", cmap='bwr',
                                vmin=-4, vmax=4,
                                min_logfoldchange=3,
                                colorbar_title='log fold change'
                               )

In [ ]:
sc.pl.rank_genes_groups_dotplot(adata, n_genes=40,
                                groups = ['4'],
                                values_to_plot="logfoldchanges", cmap='bwr',
                                vmin=-4, vmax=4,
                                min_logfoldchange=2.5,
                                colorbar_title='log fold change'
                               )

In [ ]:
cluster_anno = {
    '0':'Midbrain - Dorsal - Mab21l2+', #super clear expression https://www.sciencedirect.com/science/article/pii/S0925477399001276
    '1':'Midbrain-Hindbrain boundary - En2+',#En1/2 Fgf8/17, Sp8/9, Pax8
    '2':'Forebrain - Foxg1+',
    '3':'Mesoderm - Tbx1+',
    '4':'Hindbrain - Gbx1+',
    '5':'Mesenchyme - Frontonasal - Alx3+',
    '6':'Forebrain Dorsal - Rspo2+',
    '7':'Forebrain - Optic vesicle - Mitf+',
    '8':'Hindbrain - Casz1+',
    '9':'Mesoderm - Foxl2+',
    '10':'Surface Ectoderm - Trp63+',
    '11':'Mesenchyme - Maxillary - Dlx1+ Dlx5-',
    '12':'Mesenchyme - Mandibular - Dlx5+',
    '13':'Hindbrain - Ventral - Phox2b+',#Floor plate? part shh+, Foxa2+ part Hoxa2+ seems posterior floorplate
    '14':'Cranial Placodes - Six6+',
    '15':'Midbrain - Ventral - Foxa1+',#floor plate? shh+ and Otx2+, Dkk2 and spink1
    '16':'Forebrain - Ventral - Nkx2-4+',#Shh ventral diencephalon, Future hypothalamus? Paper in xenopus seems to indicate that Nkx2-1/4 overlap but 4 is more restricted to the ventral diencephalon
    '17':'Neural Crest - Sox10+',
    '18':'Endothelium - Emcn+',
    '19':'Neurons? - Myt1+', #myt1 neurod1 strong signal, half Otx2+ other half Hoxa2+, seems a mix of neurons from different origins
    '20':'Pharyngeal Pouch - Pax1+',
    '21':'Hindbrain - Vgll3+', #Mafb, Lmx1a, Tmem255a
    '22':'Mesoderm - Pharyngeal - Tlx1+',
    '23':'Trigeminal Ganglia - Tlx2+',
    '24':'Otic Placode - Sox10+',
    '25':'Mesoderm - Aldh1a2+',
    '26':'Myocyte - Tnni1+',
}
adata.obs['annotation'] = adata.obs['leiden_post_QC'].map(cluster_anno)

adata.obs["annotation_ordered"] = pd.Categorical(
    values=adata.obs.annotation, 
    categories=['Mesoderm - Foxl2+',
                'Mesoderm - Tbx1+',
                'Mesoderm - Pharyngeal - Tlx1+',
                'Mesoderm - Aldh1a2+',
                'Mesenchyme - Maxillary - Dlx1+ Dlx5-',
                'Mesenchyme - Mandibular - Dlx5+',
                'Mesenchyme - Frontonasal - Alx3+',
                'Neural Crest - Sox10+',
                'Myocyte - Tnni1+',
                'Surface Ectoderm - Trp63+',
                'Pharyngeal Pouch - Pax1+',
                'Cranial Placodes - Six6+',
                'Otic Placode - Sox10+',
                'Trigeminal Ganglia - Tlx2+',
                'Endothelium - Emcn+',
                'Hindbrain - Gbx1+',
                'Hindbrain - Casz1+',
                'Hindbrain - Vgll3+',
                'Hindbrain - Ventral - Phox2b+',
                'Neurons? - Myt1+',
                'Midbrain-Hindbrain boundary - En2+',
                'Midbrain - Dorsal - Mab21l2+',
                'Midbrain - Ventral - Foxa1+',
                'Forebrain Dorsal - Rspo2+',
                'Forebrain - Ventral - Nkx2-4+', 
                'Forebrain - Foxg1+',
                'Forebrain - Optic vesicle - Mitf+',
               ], 
    ordered=True
)
adata.uns['annotation_ordered_colors'] = ["#438560",
"#d7b034",
"#6fcda2",
"#62da41",
"#85722c",
"#65c263",
"#c3b96f",
"#578030",
"#a8c73e"]+["#be686d",
"#de8430",
"#d44468",
"#e09c72",
"#d84632",
"#98572f"]+["#4e519c",
"#63d6ad",
"#4c3dd4",
"#3f786b",
"#6970df",
"#85c3d1",
"#455e7e",
"#839dd8"]+["#a045cb",
"#ce99c8",
"#cf499e",
"#784371"]

sc.pl.umap(adata, color = ['annotation_ordered'])

In [ ]:
adata.write(output_prefix+species+'/'+genome+'/'+'ME9_annotated.h5ad')
adata_dict['ME9'] = adata

In [ ]:
output_prefix+species+'/'+genome+'/'+'ME9_annotated.h5ad'

In [ ]:
adata_ecto_E9 = adata[adata.obs['leiden_post_QC'].isin(['10','14','20','23','24'])].copy()

In [ ]:
sc.pp.normalize_total(adata_ecto_E9, target_sum=1e4)
sc.pp.log1p(adata_ecto_E9)
sc.pp.highly_variable_genes(adata_ecto_E9, min_disp = 0.5,
                            n_top_genes=500
                           )
sc.tl.pca(adata_ecto_E9)
sc.pp.neighbors(adata_ecto_E9, n_pcs=10)
sc.tl.umap(adata_ecto_E9,)
sc.tl.leiden(adata_ecto_E9)
sc.pl.umap(adata_ecto_E9, color = ['phase','leiden_post_QC','leiden'],size = 50, frameon=False)

In [ ]:
sc.pl.umap(adata_ecto_E9, color = ['Casr','Nkx2-6','Rnf182','Shh','Vax1','Fgf17','Amn','Tfap2b','Hapln1','Neurog1','Lmo2','Fgf8','Aldh1a3','Aldh1a7','Sp8','Wnt6','Hesx1','Foxe1','Pax6','Six1','Thrb','Sox10','Tlx2','Hmx1'],cmap = reds, vmin = 0.05, ncols = 3,size = 100, frameon=False)

In [ ]:
sc.tl.rank_genes_groups(adata_ecto_E9, groupby = 'leiden', method = 'wilcoxon')

In [ ]:
sc.pl.rank_genes_groups_dotplot(adata_ecto_E9, n_genes=8,
                                values_to_plot="logfoldchanges", cmap='bwr',
                                vmin=-4, vmax=4,
                                min_logfoldchange=2,
                                colorbar_title='log fold change'
                               )

In [ ]:
sc.pl.rank_genes_groups_dotplot(adata_ecto_E9, n_genes=30,
                                groups = ['0'],
                                values_to_plot="logfoldchanges", cmap='bwr',
                                vmin=-4, vmax=4,
                                min_logfoldchange=2.5,
                                colorbar_title='log fold change'
                               )

In [ ]:
#DEG between two clusters
cluster_of_interest = '4'
reference_cluster = '6'
cat_name = 'leiden'
sc.set_figure_params(dpi=50,figsize=[6,4])

sc.tl.rank_genes_groups(adata_ecto_E9, cat_name, groups=[cluster_of_interest], reference=reference_cluster, method='wilcoxon')


In [ ]:
df = sc.get.rank_genes_groups_df(adata_ecto_E9, group=None,)
df = df[df['logfoldchanges']>2]
sc.tl.rank_genes_groups(adata_ecto_E9, groupby = 'leiden', method = 'wilcoxon')
sc.pl.rank_genes_groups_dotplot(adata_ecto_E9,var_names = list(df['names'].head(30)),dendrogram=True,
             values_to_plot="logfoldchanges", cmap='bwr',
    vmin=-4,
    vmax=4,)


In [ ]:
adata_shark = sc.read('../../Elio/Projects/Sharks/Neural_crest/h5ad_files/shark89.h5ad')
adata_shark

In [ ]:
adata_shark.X = adata_shark.layers['original_counts'].copy()
adata_shark.write('../../Elio/Projects/Sharks/Neural_crest/h5ad_files/shark89_samap.h5ad')

In [ ]:
adata_dict['ME9'].X = adata_dict['ME9'].layers['original_counts'].copy()
adata_dict['ME9'].write(output_prefix+species+'/'+genome+'/'+'ME9_annotated_samap.h5ad')


In [ ]:
adata_dict['ME8'].X = adata_dict['ME8'].layers['original_counts'].copy()
adata_dict['ME8'].write(output_prefix+species+'/'+genome+'/'+'ME8_annotated_samap.h5ad')


In [ ]:
#ME10
adata = adata_dict['ME10']
sc.tl.leiden(adata, resolution = 1.5, key_added = 'leiden_high')


In [ ]:
sc.pl.umap(adata, color = ['leiden_post_QC','leiden_high','phase',],
           #groups = ['35'],
           legend_loc = 'on data',
           ncols = 1)

In [ ]:
sc.pl.umap(adata,color = ['Otx2','Hoxa2','Hoxb2','Dlx1','Dlx2','Dlx4','Dlx5','Dlx6','Dlx3','Alx1','Alx3','Alx4','Pitx2','Meox1','Meox2','Tnni1','Hand1','Hand2','Lhx8','Twist2','Tbx1','Tbx2','Col23a1','Colec12',
                          'Vtn','Tbx18','Tcf21','Foxd1','Foxf2','Flrt2','Pax9','Hpse2','Gm5127','Lmx1b','Lmx1a','Sema3d','Col1a1','Zic1','Rgs5','Lrba','Hoxa2','Hoxb2','Smoc2',
                          'leiden_post_QC','leiden_high'], legend_loc = 'on data',
           #['Mab21l2','Gbx2','Phox2a','Phox2b','Shh','Nkx2-1','Nkx2-4','Mitf','Vax2','Pax6','Emx2','Six6','Six3','Rspo2','Rspo3','Lhx5','Lhx2','Foxg1','Hoxa2','Otx2','Tal2','Pax3','Vgll3','Tmem255a','Plk3','Myt1','Neurod1','Aldh1a2','Dlx1','Dlx5','Trp63'],
           ncols = 3,cmap=reds, vmin =0.05)

In [ ]:
sc.pl.umap(adata, color = 'leiden_high',groups = ['31'])

In [ ]:
sc.tl.rank_genes_groups(adata, groupby = 'leiden_post_QC', method = 'wilcoxon', key_added = 'wilcoxon_leiden_post_QC')
sc.tl.rank_genes_groups(adata, groupby = 'leiden_high', method = 'wilcoxon', key_added = 'wilcoxon_leiden_high')

In [ ]:
sc.tl.dendrogram(adata, groupby='leiden_high')
sc.pl.rank_genes_groups_dotplot(adata, n_genes=4,
                                values_to_plot="logfoldchanges", cmap='bwr',
                                vmin=-4, vmax=4,
                                min_logfoldchange=3,
                                colorbar_title='log fold change'
                               )

In [ ]:
sc.pl.rank_genes_groups_dotplot(adata, n_genes=60,
                                groups = ['5'],
                                values_to_plot="logfoldchanges", cmap='bwr',
                                vmin=-4, vmax=4,
                                min_logfoldchange=2.5,
                                colorbar_title='log fold change'
                               )

In [ ]:
sc.pl.rank_genes_groups_dotplot(adata, n_genes=60,
                                groups = ['3'],
                                values_to_plot="logfoldchanges", cmap='bwr',
                                vmin=-4, vmax=4,
                                min_logfoldchange=2,
                                colorbar_title='log fold change'
                               )

In [ ]:
sc.pl.rank_genes_groups_dotplot(adata, n_genes=60,
                                groups = ['10'],
                                values_to_plot="logfoldchanges", cmap='bwr',
                                vmin=-4, vmax=4,
                                min_logfoldchange=2.5,
                                colorbar_title='log fold change'
                               )

In [ ]:
cluster_anno = {
    '5':'Mesoderm - Zic1+',
    '11':'Mesoderm? - Tbx1+',

    
}

In [ ]:
adata_ecto = adata[adata.obs['leiden_post_QC'].isin(['7','16','20'])].copy()
adata_ecto.X = adata_ecto.layers['original_counts'].copy()
sc.pp.normalize_total(adata_ecto, target_sum=1e4)
sc.pp.log1p(adata_ecto)
sc.pp.highly_variable_genes(adata_ecto, min_disp = 0.5,
                            n_top_genes=500
                           )
sc.tl.pca(adata_ecto)
sc.pp.neighbors(adata_ecto, n_pcs=10)
sc.tl.umap(adata_ecto,)
sc.tl.leiden(adata_ecto)
sc.pl.umap(adata_ecto, color = ['phase','leiden_post_QC','leiden'], frameon=False)

In [ ]:
sc.pl.umap(adata_ecto, color = ['Sfrp2','Cryab','Sox2','Foxa1','Runx1','Aldh1a7','Hmx1','Tfap2b','Thrb','Cdh6','Lgr5','Aldh1a3','Pax6','Six3','Six6','Shh','Hesx1','Six1','Sox10','Dlx2','Foxe1','Wnt6','Fgf8',],cmap = reds, vmin = 0.05, ncols =3, frameon=False)

In [ ]:
sc.tl.rank_genes_groups(adata_ecto, groupby = 'leiden', method = 'wilcoxon')

In [ ]:
sc.pl.rank_genes_groups_dotplot(adata_ecto, n_genes=8,
                                values_to_plot="logfoldchanges", cmap='bwr',
                                vmin=-4, vmax=4,
                                min_logfoldchange=2.5,
                                colorbar_title='log fold change'
                               )

In [ ]:
sc.pl.rank_genes_groups_dotplot(adata_ecto, n_genes=20,
                                groups = ['7'],
                                values_to_plot="logfoldchanges", cmap='bwr',
                                vmin=-4, vmax=4,
                                min_logfoldchange=2.5,
                                colorbar_title='log fold change'
                               )

In [ ]:
plt.rcParams['figure.figsize'] = (10, 8)
sc.pl.umap(adata,color = ['Ascl1','Rnf182','Casr','Krt8','Neurod1','Epcam','Omp','leiden_high'],legend_loc = 'on data',
           #['Mab21l2','Gbx2','Phox2a','Phox2b','Shh','Nkx2-1','Nkx2-4','Mitf','Vax2','Pax6','Emx2','Six6','Six3','Rspo2','Rspo3','Lhx5','Lhx2','Foxg1','Hoxa2','Otx2','Tal2','Pax3','Vgll3','Tmem255a','Plk3','Myt1','Neurod1','Aldh1a2','Dlx1','Dlx5','Trp63'],
           ncols = 3,cmap=reds, vmin =0.05)

In [ ]:
sc.tl.rank_genes_groups(adata, groupby = 'leiden_high', method = 'wilcoxon')

In [ ]:
sc.tl.dendrogram(adata, 'leiden_high')
sc.pl.rank_genes_groups_dotplot(adata, n_genes=80,
                                groups = ['37'],
                                values_to_plot="logfoldchanges", cmap='bwr',
                                vmin=-4, vmax=4,
                                min_logfoldchange=4,
                                colorbar_title='log fold change'
                               )

In [ ]:
#ME11
adata = adata_dict['ME11']
sc.tl.leiden(adata, resolution = 3, key_added = 'leiden_high')


In [ ]:
sc.pl.umap(adata, color = ['leiden_post_QC','leiden_high'],
           #groups = ['35'],
           legend_loc = 'on data',
           ncols = 1)

In [ ]:
plt.rcParams['figure.figsize'] = (10, 8)

sc.pl.umap(adata,color = ['Casr','Rnf182','Krt8','Epcam','Neurod1','Neurog1','Ascl1'],
           #['Mab21l2','Gbx2','Phox2a','Phox2b','Shh','Nkx2-1','Nkx2-4','Mitf','Vax2','Pax6','Emx2','Six6','Six3','Rspo2','Rspo3','Lhx5','Lhx2','Foxg1','Hoxa2','Otx2','Tal2','Pax3','Vgll3','Tmem255a','Plk3','Myt1','Neurod1','Aldh1a2','Dlx1','Dlx5','Trp63'],
           ncols = 3,cmap=reds, vmin =0.05)

In [ ]:
#ME12
adata = adata_dict['ME12']
sc.tl.leiden(adata, resolution = 3, key_added = 'leiden_high')
sc.pl.umap(adata, color = ['leiden_post_QC','leiden_high'],
           #groups = ['35'],
           legend_loc = 'on data',
           ncols = 1)

In [ ]:
sc.pl.umap(adata,color = ['Casr','Rnf182','Krt8','Epcam','Neurod1','Neurog1','Ascl1'],
           #['Mab21l2','Gbx2','Phox2a','Phox2b','Shh','Nkx2-1','Nkx2-4','Mitf','Vax2','Pax6','Emx2','Six6','Six3','Rspo2','Rspo3','Lhx5','Lhx2','Foxg1','Hoxa2','Otx2','Tal2','Pax3','Vgll3','Tmem255a','Plk3','Myt1','Neurod1','Aldh1a2','Dlx1','Dlx5','Trp63'],
           ncols = 3,cmap=reds, vmin =0.05)